In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function
from scipy.interpolate import griddata

In [2]:
def enum_other_states(n, min_s, max_s):
    """enumerate the state. for all palyers except player i
    (1,1,2) is equivalent to (1,2,1) from firm 1's pespective"""
    
    states_i = range(min_s,max_s)
    if n <=1:
        states = [[i] for i in states_i]
        return states
    else:
        states = []
        states_j = enum_other_states(n-1,min_s,max_s)
        for i in states_i:
            for j in states_j:
                if j[0] >= i:
                    state_ij = [i] + j 
                    states.append(state_ij)
        return states

In [3]:
class States:
    """class for dealing with the states
    these things are apparently really complex
    there are just a ton of them"""
    
    my_states = None #type nparray
    other_states = None #type np array 2d
    n = 0 #number of players
    max_states = 0
    min_states = 0
    

    def __init__(self, n, min_s,max_s):
        """initialize important class attributes"""
        self.n, self.min_states, self.max_states = n, min_s, max_s
      
    
    def gen_states(self):
        """compute all relevant states for firm i
        the trick is ommiting equivalent states
        this is taken care of in the helper function"""
        
        #avoid typing self a million times
        n, min_s, max_s  = self.n, self.min_states, self.max_states
        
        #setup other states
        other_states =  np.array(enum_other_states(n-1,min_s,max_s))
        num_others = other_states.shape[0]
        
        #initialize my states based on this
        my_states = np.arange(min_s,max_s)
        my_states = np.repeat(my_states,num_others )
        self.my_states = my_states
        
        #copy other states enough times
        other_states = other_states.reshape((1,num_others,n-1))
        other_states = np.tile(other_states,(max_s - min_s,1,1))  
        other_states = other_states.reshape(num_others*(max_s - min_s),n-1)
        self.other_states = other_states
    
    
    
    def make_states(self, my_states, other_states):
        """special initializer for states
        ensures they are not too big or too small"""
        
        assert np.array(other_states).shape[1] == (self.n -1)
        
        n, min_s, max_s  = self.n, self.min_states, self.max_states
        #make a copy to avoid side effects
        new_states = States(n,min_s,max_s)
        
        #ensure other my states are good
        my_states = np.array(my_states)
        my_states = np.minimum(max_s - 1, np.array(my_states))
        new_states.my_states = np.maximum(min_s, np.array(my_states))
        
        #ensure other states are good
        other_states = np.sort(other_states,axis=1)
        other_states = np.minimum(max_s - 1, np.array(other_states))
        new_states.other_states = np.maximum(min_s, np.array(other_states))
        
        return new_states
    
    
    def get_len(self):
        return len(self.other_states)
    
    
    def get_all(self):
        """return all the states"""
        reshaped_states = self.my_states.reshape((len(self.my_states),1))
        return np.concatenate( (reshaped_states, self.other_states), axis=1 )

In [4]:
def create_function(states, values):
    """returns a function
    the function uses the grid data to figure out return value"""
    f = lambda eval_states: griddata(states.get_all(), values, 
                                     eval_states.get_all(), method='linear')
    return f

In [5]:
def calc_bids(policy, states):
    """return the other players bids
    given the state by 'switching perspective'
    to other firm"""
    
    other_bids = []
    for i in range(1,states.n):
        other_states = states.get_all()
        my_other = other_states[:,i]
        other = np.delete(other_states, i, axis=1)
        other_states = states.make_states(my_other, other)
        other_bids.append( policy(other_states) )
    return np.array(other_bids).transpose()

In [55]:
def cost(states, shock,theta):
    """return the cost for firm 0"""
    return theta*states.my_states + shock


def profit(bids, states, contract, shock, theta):
    """return firm 0's profit"""
    my_bids = bids[:,0]
    other_bids = np.delete(bids,0,axis=1)
    profits = contract*( my_bids < np.min(other_bids, axis=1) )*(my_bids - cost(states,shock,theta))  
    return profits


def update_state(bids, states, contract):
    """update all firms states based on bids"""
    new_states = contract*(  bids< np.min(bids) ) + states.get_all()
    return states.make_states(new_states[:,0],np.delete(new_states,0,axis=1))


##### unit testing

#make some bids
bids = np.array([1,2,3])

#make some states
states = States(3,1,3)
states.gen_states()
new_states = states.make_states([1,1,2],[[1.5,1],[1,5],[1,4.3]])

#make a policy
values = np.concatenate([6*np.ones(states.get_len()/2),8*np.ones(states.get_len()/2)])
policy = create_function(states, values)

#learn mkt outcome for others
other_bids = calc_bids(policy, new_states)
all_bids = bids.reshape(len(bids),1)
all_bids = np.concatenate((all_bids,other_bids), axis=1)

#tests
print cost(new_states,0,1.1)
print profit(all_bids,new_states,1,0,1.1)
print update_state(all_bids,new_states,1.1).get_all()

[1.1 1.1 2.2]
[-0.1  0.9  0.8]
[[1.  1.  1.5]
 [1.  1.  2. ]
 [2.  1.  2. ]]


In [64]:
def operator(v_init, p_init, bids, states, theta, beta):
    """update the value and the policy function for player 0
    
    since we are looking for symmetric eq, there is really just 1
    policy and value to keep track of"""
    
    #initialize contract and state to 0
    contract = 1
    shock = 0
    
    #repeat bids/states per each bid
    my_bids = np.repeat(bids,states.get_len())
    bid_states = states.get_all()
    bid_states = np.tile(bid_states,(len(bids),1))
    bid_states = states.make_states(bid_states[:,0],np.delete(bid_states,0,axis=1))
    
    #other bids in each bid state and concat with my bids
    other_bids = calc_bids(p_init, bid_states)
    all_bids = my_bids.reshape(len(my_bids),1)
    all_bids = np.concatenate((all_bids,other_bids), axis=1)
    
    #calculate see how the states would update
    next_states = update_state(all_bids, bid_states, contract)
    
    #calculate the value of the next state
    v_new = profit(all_bids, bid_states, contract,  shock, theta)
    v_new = v_new + beta*v_init(next_states)
    
    #reshape v_new to find argmax
    v_new = v_new.reshape((len(bids),states.get_len()))
    
    policy = np.argmax(v_new,axis=1)
    policy = bids[policy]
    policy = create_function(states, policy)
    
    value = create_function(states, v_new)
    
    return policy, value

#make some states/bids
bids = np.arange(1,6)
states = States(3,1,3)
states.gen_states()

#make a policy
values = np.concatenate([6*np.ones(states.get_len()/2),8*np.ones(states.get_len()/2)])
policy = create_function(states, values)

policy, value = operator(policy, policy, bids, states, 1.1, 0)

In [ ]:
def graph_functions():

In [ ]:
def  markov_perfect_eq(initial_value, error, maxiter):
    """calculate the value function in a symmetric
        markov perfect equilibria"""
    
    #initialize loop variables
    v0 = np.ones(len(states))
    v_next = operator(a, theta1, cost, v0, v1)
    
    while  maxiter >= 0 and ( 
            np.abs(v0 - v0_next ).max() > error
           or np.abs(v1 - v1_next ).max()  > error ):
        
        #iterate loop variables for each player
        for i in range(N):
            v0, v1 = v0_next, v1_next
            v0_next, v1_next = operator(a, theta1, cost, v0, v1)
            maxiter = maxiter -1

    return  value